# Find calendar

### for experiments where I have made a global annual average

In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import intake
import importlib
import my_functions as f
importlib.reload(f) # needed to load new changes in my_functions


<module 'my_functions' from '/Users/hege-beatefredriksen/OneDrive - UiT Office 365/Papers/ForcingCMIP6/CMIP6-forcing/notebooks/my_functions.py'>

In [2]:
# check which folders are available, and make a dictionary of available experiments

experiments = {}
directory = '../Processed_data/Global_annual_means/'
model_names = [ f.name for f in os.scandir(directory) if f.is_dir() and f.name !='.ipynb_checkpoints']
model_names.sort()

for model in model_names:
    #print(model)
    experiments[model] = {}
    modeldirectory = os.path.join(directory, model)
    modelexp_names = [ f.name for f in os.scandir(modeldirectory) if f.is_dir() and f.name !='.ipynb_checkpoints']
    for exp in modelexp_names:
        #print(exp)
        experiments[model][exp] = f.find_members(model, exp, datatype = 'means')


In [6]:
model_names;

## Select model(s)

In [5]:
#selected_models = model_names
#selected_models = ['MPI-ESM-1-2-HAM']
selected_models = ['CanESM5', 'GISS-E2-1-G', 'IPSL-CM6A-LR', 'MIROC6']
experiments[selected_models[-1]]

{'piClim-control': ['r11i1p1f1', 'r1i1p1f1'],
 'ssp585': ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
 'abrupt-2xCO2': ['r1i1p1f1'],
 'abrupt-0p5xCO2': ['r1i1p1f1'],
 'piControl': ['r1i1p1f1'],
 'abrupt-4xCO2': ['r1i1p1f1'],
 'ssp245': ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
 'historical': ['r10i1p1f1',
  'r1i1p1f1',
  'r2i1p1f1',
  'r3i1p1f1',
  'r4i1p1f1',
  'r5i1p1f1',
  'r6i1p1f1',
  'r7i1p1f1',
  'r8i1p1f1',
  'r9i1p1f1'],
 'piClim-4xCO2': ['r1i1p1f1'],
 '1pctCO2': ['r1i1p1f1'],
 'piClim-histall': ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
 'ssp370': ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
 'ssp126': ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
 'ssp119': ['r1i1p1f1']}

In [4]:
col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)

In [6]:
# loop through models instead
    
#for model in model_names:
for model in selected_models:
    filename = '../Processed_data/Calendars/' + model + '_calendars.txt'
    column_names = ['model', 'exp', 'member', 'calendar']
    df = pd.DataFrame(columns = column_names)

    #for model in experiments:
    for exp in experiments[model]:
            for member in experiments[model][exp]:
                cat = col.search(experiment_id = exp, variable_id='tas', table_id='Amon', source_id = model, member_id = member)
                dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
                for key in dset_dict.keys():
                    ds = dset_dict[key]

                calendar = ds.time.encoding['calendar']
                print(model, exp, member, calendar)
                df_row = pd.DataFrame([[model, exp, member, calendar]] ,columns = column_names)
                df = df.append(df_row, ignore_index = True)
    df.to_csv(filename, ' ')
                



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


CanESM5 piClim-control r1i1p2f1 noleap

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


CanESM5 ssp585 r10i1p1f1 365_day

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


CanESM5 ssp585 r10i1p2f1 365_day

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


CanESM5 ssp585 r11i1p1f1 365_day

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


...

MIROC6 ssp370 r3i1p1f1 gregorian

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


MIROC6 ssp126 r1i1p1f1 gregorian

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


MIROC6 ssp126 r2i1p1f1 gregorian

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


MIROC6 ssp126 r3i1p1f1 gregorian

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


MIROC6 ssp119 r1i1p1f1 gregorian
